In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns

In [12]:
# Load and explore the data
df = pd.read_csv("../data/raw-rain-data.csv")
print("Dataset shape:", df.shape)
print("\nColumn names:", df.columns.tolist())
print("\nData types:")
print(df.dtypes)
print("\nFirst few rows:")
print(df.head())

Dataset shape: (7084, 7)

Column names: ['YEAR', 'MONTH', 'PROV_ID', 'PROV_T', 'MinRain', 'MaxRain', 'AvgRain']

Data types:
YEAR         int64
MONTH        int64
PROV_ID      int64
PROV_T      object
MinRain    float64
MaxRain    float64
AvgRain    float64
dtype: object

First few rows:
   YEAR  MONTH  PROV_ID           PROV_T    MinRain     MaxRain     AvgRain
0  2018      1       10    กรุงเทพมหานคร  54.299999  257.230011  142.119137
1  2018      1       11      สมุทรปราการ  76.250000  256.100006  137.302046
2  2018      1       12          นนทบุรี  38.360001  161.470001  113.433771
3  2018      1       13         ปทุมธานี  51.439999  116.500000   82.901688
4  2018      1       14  พระนครศรีอยุธยา   8.850000   88.589996   39.960089


In [13]:
# Check for missing values and basic statistics
print("Missing values:")
print(df.isnull().sum())
print("\nBasic statistics:")
df.describe(include='all')

Missing values:
YEAR       0
MONTH      0
PROV_ID    0
PROV_T     0
MinRain    0
MaxRain    0
AvgRain    0
dtype: int64

Basic statistics:


,YEAR,MONTH,PROV_ID,PROV_T,MinRain,MaxRain,AvgRain
count,7084.000000,7084.000000,7084.000000,7084,7084.000000,7084.000000,7084.000000
unique,NaN,NaN,NaN,77,NaN,NaN,NaN
top,NaN,NaN,NaN,กรุงเทพมหานคร,NaN,NaN,NaN
freq,NaN,NaN,NaN,92,NaN,NaN,NaN
mean,2021.347826,6.326087,51.129870,NaN,85.751294,190.774180,130.285111
std,2.218826,3.414115,24.943954,NaN,99.093571,189.948223,132.369089
min,2018.000000,1.000000,10.000000,NaN,0.000000,0.000000,0.000000
25%,2019.000000,3.000000,31.000000,NaN,4.765000,37.292500,18.271285
50%,2021.000000,6.000000,50.000000,NaN,56.670000,152.930000,102.885111
75%,2023.000000,9.000000,72.000000,NaN,135.262496,278.827507,198.167024


In [14]:
# Data Quality Check
print("Unique provinces:", df['PROV_T'].nunique())
print("Year range:", df['YEAR'].min(), "to", df['YEAR'].max())
print("Month range:", df['MONTH'].min(), "to", df['MONTH'].max())
print("\nProvince distribution:")
print(df['PROV_T'].value_counts().head(10))

Unique provinces: 77
Year range: 2018 to 2025
Month range: 1 to 12

Province distribution:
PROV_T
กรุงเทพมหานคร      92
สมุทรปราการ        92
นนทบุรี            92
ปทุมธานี           92
พระนครศรีอยุธยา    92
อ่างทอง            92
ลพบุรี             92
สิงห์บุรี          92
ชัยนาท             92
สระบุรี            92
Name: count, dtype: int64


## Data Preprocessing Pipeline

### 1. Handle Missing Values and Outliers

In [15]:
# Create a copy for preprocessing
df_processed = df.copy()

# Check for negative rainfall values (data quality issue)
print("Negative rainfall values:")
print("MinRain:", (df_processed['MinRain'] < 0).sum())
print("MaxRain:", (df_processed['MaxRain'] < 0).sum())
print("AvgRain:", (df_processed['AvgRain'] < 0).sum())

# Handle negative values by setting them to 0 (assuming they're measurement errors)
rainfall_cols = ['MinRain', 'MaxRain', 'AvgRain']
for col in rainfall_cols:
    df_processed[col] = df_processed[col].clip(lower=0)

# Handle outliers using IQR method
def remove_outliers_iqr(df, columns):
    df_clean = df.copy()
    for col in columns:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df_clean[col] = df_clean[col].clip(lower=lower_bound, upper=upper_bound)
    return df_clean

# Optional: Apply outlier removal (uncomment if needed)
# df_processed = remove_outliers_iqr(df_processed, rainfall_cols)

print("\nData shape after cleaning:", df_processed.shape)

Negative rainfall values:
MinRain: 0
MaxRain: 0
AvgRain: 0

Data shape after cleaning: (7084, 7)


### 2. Feature Engineering

In [16]:
# Create season feature
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

df_processed['Season'] = df_processed['MONTH'].apply(get_season)

# Create rainfall range feature
df_processed['RainRange'] = df_processed['MaxRain'] - df_processed['MinRain']

# Create rainfall variability feature (coefficient of variation approximation)
df_processed['RainVariability'] = df_processed['RainRange'] / (df_processed['AvgRain'] + 1e-6)

# Create categorical features for time
df_processed['Quarter'] = ((df_processed['MONTH'] - 1) // 3) + 1
df_processed['IsRainySeason'] = df_processed['MONTH'].apply(lambda x: 1 if x in [5, 6, 7, 8, 9, 10] else 0)

print("New features created:")
print("Season:", df_processed['Season'].value_counts())
print("\nQuarter:", df_processed['Quarter'].value_counts())
print("\nRainy Season distribution:", df_processed['IsRainySeason'].value_counts())

New features created:
Season: Season
Spring    1848
Summer    1848
Winter    1771
Fall      1617
Name: count, dtype: int64

Quarter: Quarter
1    1848
2    1848
3    1771
4    1617
Name: count, dtype: int64

Rainy Season distribution: IsRainySeason
0    3542
1    3542
Name: count, dtype: int64


### 3. Encoding Categorical Variables

#### One-Hot Encoding for Categorical Features

In [17]:
# Store original columns before encoding
original_cols = df_processed.columns.tolist()

# One-hot encode province names (PROV_T)
print("Applying one-hot encoding to provinces...")
province_encoded = pd.get_dummies(df_processed['PROV_T'], prefix='Province', drop_first=True)
df_processed = pd.concat([df_processed, province_encoded], axis=1)

# One-hot encode seasons
season_encoded = pd.get_dummies(df_processed['Season'], prefix='Season', drop_first=True)
df_processed = pd.concat([df_processed, season_encoded], axis=1)

# One-hot encode quarters
quarter_encoded = pd.get_dummies(df_processed['Quarter'], prefix='Q', drop_first=True)
df_processed = pd.concat([df_processed, quarter_encoded], axis=1)

# Drop original categorical columns
df_processed = df_processed.drop(['PROV_T', 'Season', 'Quarter'], axis=1)

print(f"Dataset shape after one-hot encoding: {df_processed.shape}")
print(f"New columns added: {df_processed.shape[1] - len(original_cols) + 3}")  # +3 for dropped columns

Applying one-hot encoding to provinces...
Dataset shape after one-hot encoding: (7084, 91)
New columns added: 82


#### Label Encoding (Alternative approach)

In [18]:
# Create a separate version with label encoding for comparison
df_label_encoded = df.copy()

# Apply label encoding to provinces
le_province = LabelEncoder()
df_label_encoded['PROV_T_encoded'] = le_province.fit_transform(df_label_encoded['PROV_T'])

# Add other features
df_label_encoded['Season'] = df_label_encoded['MONTH'].apply(get_season)
le_season = LabelEncoder()
df_label_encoded['Season_encoded'] = le_season.fit_transform(df_label_encoded['Season'])

print("Label encoding completed")
print("Province encoding mapping (first 5):")
for i, province in enumerate(le_province.classes_[:5]):
    print(f"{province}: {i}")

Label encoding completed
Province encoding mapping (first 5):
กระบี่: 0
กรุงเทพมหานคร: 1
กาญจนบุรี: 2
กาฬสินธุ์: 3
กำแพงเพชร: 4


### 4. Feature Scaling/Normalization

In [19]:
required_base = ['MinRain', 'MaxRain', 'AvgRain']
missing_base = [c for c in required_base if c not in df_processed.columns]
if missing_base:
    raise KeyError(f"Missing base columns: {missing_base}")

if 'RainRange' not in df_processed.columns:
    df_processed['RainRange'] = df_processed['MaxRain'] - df_processed['MinRain']

if 'RainVariability' not in df_processed.columns:
    eps = 1e-9
    df_processed['RainVariability'] = (df_processed['MaxRain'] - df_processed['MinRain']) / (df_processed['AvgRain'].abs() + eps)

numerical_features = ['MinRain', 'MaxRain', 'AvgRain', 'RainRange', 'RainVariability']

df_processed[numerical_features] = df_processed[numerical_features].replace([float('inf'), float('-inf')], None)
df_processed[numerical_features] = df_processed[numerical_features].fillna(0)

print("Applying MinMax scaling...")
scaler_minmax = MinMaxScaler()
df_processed[numerical_features] = scaler_minmax.fit_transform(df_processed[numerical_features])

df_standard = df_processed.copy()
scaler_standard = StandardScaler()
df_standard[numerical_features] = scaler_standard.fit_transform(df_standard[numerical_features])

print("Scaling completed")
print("\nScaled data summary (MinMax):")
print(df_processed[numerical_features].describe())

Applying MinMax scaling...
Scaling completed

Scaled data summary (MinMax):
           MinRain      MaxRain      AvgRain    RainRange  RainVariability
count  7084.000000  7084.000000  7084.000000  7084.000000      7084.000000
mean      0.067834     0.116159     0.089689     0.109820         0.001278
std       0.078388     0.115656     0.091124     0.124223         0.017927
min       0.000000     0.000000     0.000000     0.000000         0.000000
25%       0.003769     0.022707     0.012578     0.026100         0.000214
50%       0.044829     0.093117     0.070827     0.074970         0.000359
75%       0.107000     0.169773     0.136420     0.146933         0.000645
max       1.000000     1.000000     1.000000     1.000000         1.000000


### 5. Final Dataset Overview

In [20]:
print("=== FINAL PREPROCESSED DATASET ===")
print(f"Shape: {df_processed.shape}")
print(f"Columns: {len(df_processed.columns)}")
print("\nColumn names:")
for i, col in enumerate(df_processed.columns):
    print(f"{i+1:2d}. {col}")

print(f"\nMemory usage: {df_processed.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("\nFirst few rows of processed data:")
df_processed.head()

=== FINAL PREPROCESSED DATASET ===
Shape: (7084, 91)
Columns: 91

Column names:
 1. YEAR
 2. MONTH
 3. PROV_ID
 4. MinRain
 5. MaxRain
 6. AvgRain
 7. RainRange
 8. RainVariability
 9. IsRainySeason
10. Province_กรุงเทพมหานคร
11. Province_กาญจนบุรี
12. Province_กาฬสินธุ์
13. Province_กำแพงเพชร
14. Province_ขอนแก่น
15. Province_จันทบุรี
16. Province_ฉะเชิงเทรา
17. Province_ชลบุรี
18. Province_ชัยนาท
19. Province_ชัยภูมิ
20. Province_ชุมพร
21. Province_ตรัง
22. Province_ตราด
23. Province_ตาก
24. Province_นครนายก
25. Province_นครปฐม
26. Province_นครพนม
27. Province_นครราชสีมา
28. Province_นครศรีธรรมราช
29. Province_นครสวรรค์
30. Province_นนทบุรี
31. Province_นราธิวาส
32. Province_น่าน
33. Province_บึงกาฬ
34. Province_บุรีรัมย์
35. Province_ปทุมธานี
36. Province_ประจวบคีรีขันธ์
37. Province_ปราจีนบุรี
38. Province_ปัตตานี
39. Province_พระนครศรีอยุธยา
40. Province_พะเยา
41. Province_พังงา
42. Province_พัทลุง
43. Province_พิจิตร
44. Province_พิษณุโลก
45. Province_ภูเก็ต
46. Province_มหาสารคา

,YEAR,MONTH,PROV_ID,MinRain,MaxRain,AvgRain,RainRange,RainVariability,IsRainySeason,Province_กรุงเทพมหานคร,...,Province_เพชรบูรณ์,Province_เลย,Province_แพร่,Province_แม่ฮ่องสอน,Season_Spring,Season_Summer,Season_Winter,Q_2,Q_3,Q_4
0,2018,1,10,0.042954,0.156623,0.097836,0.212199,0.000553,0,True,...,False,False,False,False,False,False,True,False,False,False
1,2018,1,11,0.060318,0.155935,0.094520,0.188065,0.000507,0,False,...,False,False,False,False,False,False,True,False,False,False
2,2018,1,12,0.030345,0.098316,0.078089,0.128733,0.000420,0,False,...,False,False,False,False,False,False,True,False,False,False
3,2018,1,13,0.040692,0.070935,0.057070,0.068032,0.000304,0,False,...,False,False,False,False,False,False,True,False,False,False
4,2018,1,14,0.007001,0.053941,0.027509,0.083382,0.000772,0,False,...,False,False,False,False,False,False,True,False,False,False


### 6. Data Splitting for ML

In [21]:
# Prepare features (X) and target (y) - assuming AvgRain is the target
feature_cols = [col for col in df_processed.columns if col not in ['AvgRain', 'YEAR', 'MONTH', 'PROV_ID']]
X = df_processed[feature_cols]
y = df_processed['AvgRain']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=None)

print("Data splitting completed:")
print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"Feature columns ({len(feature_cols)}):")
for i, col in enumerate(feature_cols[:10]):  # Show first 10
    print(f"  {i+1}. {col}")
if len(feature_cols) > 10:
    print(f"  ... and {len(feature_cols) - 10} more")

# Save processed datasets
print("\nSaving processed datasets...")
df_processed.to_csv('data/processed_rain_data.csv', index=False)
print("Saved: data/processed_rain_data.csv")


Data splitting completed:
Training set: (5667, 87)
Test set: (1417, 87)
Feature columns (87):
  1. MinRain
  2. MaxRain
  3. RainRange
  4. RainVariability
  5. IsRainySeason
  6. Province_กรุงเทพมหานคร
  7. Province_กาญจนบุรี
  8. Province_กาฬสินธุ์
  9. Province_กำแพงเพชร
  10. Province_ขอนแก่น
  ... and 77 more

Saving processed datasets...
Saved: data/processed_rain_data.csv
